In [3]:
import pandas as pd
import numpy as np
import pickle
import json

In [2]:
structures = pd.read_csv('../Data/structures.csv')

In [4]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [21]:
atom_list = ['H','C','N','O','F']

In [22]:
with open('../Data/elemental_embedding_1MEGNet_layer.json') as f:
    atom = json.load(f)

In [23]:
atom = {a:atom[a] for a in atom_list}

In [25]:
atomic_radius = {'H':0.38+0.05, 'C':0.77+0.05, 'N':0.75+0.05, 'O':0.73+0.05, 'F':0.71+0.05} 
electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

In [26]:
for a in atom_list:
    atom[a].extend([atomic_radius[a],electronegativity[a]])

In [28]:
atom = pd.DataFrame(atom).T

In [29]:
atom

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
H,0.352363,0.635952,0.217338,-0.191956,0.253751,-0.423261,0.221297,-0.452411,-1.007713,-0.289936,0.126820,-0.025928,0.717508,-0.631993,0.021095,-0.270104,0.43,2.20
C,0.398148,0.744485,0.662636,-0.520578,-0.673885,-0.476654,0.698720,0.607385,-0.562160,0.673558,0.587440,0.619206,0.615198,-0.749764,-0.552250,-0.652751,0.82,2.55
N,0.611494,0.001810,0.620457,-0.446009,0.094431,-0.817922,-0.130236,-0.645908,0.021001,-0.271362,-0.204580,0.586720,0.663307,-0.584931,-0.367534,-0.559885,0.80,3.04
O,-0.113972,-0.188673,0.108998,0.214548,0.371144,-0.103028,-0.404307,-0.502800,0.313225,-0.465547,-0.369454,-0.103702,-0.042888,0.087557,0.192867,0.038902,0.78,3.44
F,-0.308105,-0.575614,-0.171835,0.462324,0.702146,0.220106,-0.503306,-0.394980,0.283450,-0.584290,-0.465051,-0.447120,-0.242727,0.444719,0.501850,0.371508,0.76,3.98


In [30]:
structures = structures.merge(atom,'left',left_on='atom',right_index=True)

In [32]:
atom_qm9 = pd.read_pickle('../Data/structure_qm9.pkl')

In [36]:
atom_qm9.head()

,molecule_name,atom_index,mulliken
0,dsgdb9nsd_095219,0,-0.400745
1,dsgdb9nsd_095219,1,0.096697
2,dsgdb9nsd_095219,2,-0.259809
3,dsgdb9nsd_095219,3,-0.120536
4,dsgdb9nsd_095219,4,0.052314


In [37]:
structures = structures.merge(atom_qm9,'left',on=['molecule_name','atom_index'])

In [41]:
structures = structures.sort_values(by=['molecule_name', 'atom_index'])

In [42]:
structures_gb = structures.groupby(['molecule_name'])

In [43]:
structures_dict = {k: v.iloc[:,6:].values.astype(np.float32) for k,v in structures_gb}

In [50]:
with open('../Data/structures_dict_atomInfo.pickle', 'wb') as handle:
    pickle.dump(structures_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)